In [10]:
from CoinSimulator import CoinSimulator
import sys
import logging
logging.basicConfig(filename=f"{__name__}.log", level=logging.ERROR, format='%(asctime)s - %(levelname)s - %(message)s')
def sim_test(test_id="TEST"):
    logging.info(f"\n### coins ### {test_id} ######################")
def is_positive_int(value):
    try:
        num = int(value)
        return num > 0
    except ValueError:
        return False

def coin_simulator(number_of_bits=1, number_of_tests=10):
    sim_test("COIN SIMULATOR")
    results = CoinSimulator(number_of_bits).simulate(number_of_tests)
    logging.debug(f"Results: {results}")
    for state, result in results.items():
        print(f"state: {state} result: {result}")

if __name__ == "__main__":
    sim_test("ARGUMENTS")
    logging.info(f"Number of arguments: {len(sys.argv)}")
    logging.info(f"Argument List: {sys.argv}")

    try:
        logging.debug("sys.argv[1]:"+sys.argv[1])
        logging.debug("sys.argv[2]:"+sys.argv[2])
        NUMBER_OF_OBJECTS = int(sys.argv[1])
        NUMBER_OF_SIMULATIONS = int(sys.argv[2])
    except:
        NUMBER_OF_OBJECTS = 3
        NUMBER_OF_SIMULATIONS = 10
    logging.info(f"NUMBER_OF_OBJECTS: {NUMBER_OF_OBJECTS}")
    logging.info(f"NUMBER_OF_SIMULATIONS: {NUMBER_OF_SIMULATIONS}")

    coin_simulator(NUMBER_OF_OBJECTS,NUMBER_OF_SIMULATIONS)



state: coin-0-HEADS result: 6
state: coin-0-TAILS result: 4
state: coin-1-HEADS result: 6
state: coin-1-TAILS result: 4
state: coin-2-HEADS result: 2
state: coin-2-TAILS result: 8
